In [5]:
import pandas as pd
import numpy as np
import yfinance as yf

In [193]:
def binomial_price(u,d,T,n,r,s_0,k,type, instrument):
    """"
    binomial option pricing w/ N steps for time period T 
    ----------------
    Type: str
    c or p
    ----------------
    Instrument: str
    european or american 
    ----------------
    returns np.array of option price at each times step
    """
    dt = T/n # time delta 
    q =((1 +r*dt) - d) /(u-d) #risk free probability
    discount = (1+r*dt)**-1 #discount factor
    price = s_0* d **(np.arange(n,-1,-1)) * u ** (np.arange(0,n+1,1))
    option_p = np.zeros(n+1)

    #pay offs
    for i in range(0,n+1):
        if type == 'c': #if call then max{S-K,0}
            option_p[i] = max(price[i]-k,0)
        else: #         #if put then max{k-S,0}
            option_p[i] = max(k-price[i],0)
    for i in np.arange(n-1,-1,-1): #from the n-1 time to -1 (python indexing, so zero, by one time step)

        s_node = s_0* d **(np.arange(i,-1,-1)) * u ** (np.arange(0,i+1,1)) #prices at everytime step
        option_p[0:i+1] = discount*(q*option_p[1:i+2] + (1-q)*option_p[0:i+1]) #discounts back
        option_p =option_p[:-1]
        
        if instrument == 'european': #if european then discount is the the price
            pass
        elif instrument == 'american': #if american, then at every time max(pay off and stock)
            if type == 'c':
                option_p = np.maximum(option_p,s_node-k)
                if i <10:
                    print(f"Time {i} {type} Early Exercise:", option_p > (s_node -k)) #print exercise decisions in the first 10 periods
            else:
                option_p = np.maximum(option_p,k-s_node)
                if i < 10:
                    print(f"Time {i} {type} Early Exercise:", option_p > (s_node -k))
    return option_p

### Problem 1: American options with N large

Note: This problem is a continuation of Problem 3 in HW #3. The objective is to
generalize your code to American options. Please provide the answers to the questions
below as well as a sample of your code.

We consider the same market model as in Problem 3 of HW #3. We consider an
American put option with strike price $K = 1025$ and maturity $T = 1$ year.

#### (a) Find the price of the option at time $t_0 = 0$ if it is American and compare with the price of the European put found in Problem 1.


In [194]:
call_american = binomial_price(type='c',k = 1025,T = 1,instrument = 'american',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = .05)[0]
put_american = binomial_price(type='p',k = 1025,T = 1,instrument = 'american',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = .05)[0]
call_euro = binomial_price(type='c',k = 1025,T = 1,instrument = 'european',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = .05)[0]
put_euro = binomial_price(type='p',k = 1025,T = 1,instrument = 'european',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = .05)[0]

Time 9 c Early Exercise: [ True  True  True  True  True  True  True  True  True  True]
Time 8 c Early Exercise: [ True  True  True  True  True  True  True  True  True]
Time 7 c Early Exercise: [ True  True  True  True  True  True  True  True]
Time 6 c Early Exercise: [ True  True  True  True  True  True  True]
Time 5 c Early Exercise: [ True  True  True  True  True  True]
Time 4 c Early Exercise: [ True  True  True  True  True]
Time 3 c Early Exercise: [ True  True  True  True]
Time 2 c Early Exercise: [ True  True  True]
Time 1 c Early Exercise: [ True  True]
Time 0 c Early Exercise: [ True]
Time 9 p Early Exercise: [ True  True  True  True  True  True  True False False False]
Time 8 p Early Exercise: [ True  True  True  True  True  True  True False False]
Time 7 p Early Exercise: [ True  True  True  True  True  True  True False]
Time 6 p Early Exercise: [ True  True  True  True  True  True False]
Time 5 p Early Exercise: [ True  True  True  True  True  True]
Time 4 p Early Exercise: 

In [232]:
assert call_american == call_euro
print(f'Is american put price equal to euro: {put_american == put_euro}')

Is american put price equal to euro: False



#### (b) What is the earliest possible time of optimal exercise of the American put ?


In [227]:
put_american = binomial_price(type='p',k = 1025,T = 1,instrument = 'american',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = .05)[0]
print('\nTime 6 is the earliest time to exercise')

Time 9 p Early Exercise: [ True  True  True  True  True  True  True False False False]
Time 8 p Early Exercise: [ True  True  True  True  True  True  True False False]
Time 7 p Early Exercise: [ True  True  True  True  True  True  True False]
Time 6 p Early Exercise: [ True  True  True  True  True  True False]
Time 5 p Early Exercise: [ True  True  True  True  True  True]
Time 4 p Early Exercise: [ True  True  True  True  True]
Time 3 p Early Exercise: [ True  True  True  True]
Time 2 p Early Exercise: [ True  True  True]
Time 1 p Early Exercise: [ True  True]
Time 0 p Early Exercise: [ True]

Time 6 is the earliest time to exercise



#### (c) Repeat questions (a) and (b) in the case where $r = 0$. What do you conclude ?

In [239]:
call_american_r0 = binomial_price(type='c',k = 1025,T = 1,instrument = 'american',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = 0)[0]
put_american_r0 = binomial_price(type='p',k = 1025,T = 1,instrument = 'american',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = 0)[0]
call_euro_r0 = binomial_price(type='c',k = 1025,T = 1,instrument = 'european',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = 0)[0]
put_euro_r0 = binomial_price(type='p',k = 1025,T = 1,instrument = 'european',n=60,u = 1+.005,d = 1-.003,s_0 = 1000,r = 0)[0]

Time 9 c Early Exercise: [ True  True  True  True  True  True  True  True  True  True]
Time 8 c Early Exercise: [ True  True  True  True  True  True  True  True  True]
Time 7 c Early Exercise: [ True  True  True  True  True  True  True  True]
Time 6 c Early Exercise: [ True  True  True  True  True  True  True]
Time 5 c Early Exercise: [ True  True  True  True  True  True]
Time 4 c Early Exercise: [ True  True  True  True  True]
Time 3 c Early Exercise: [ True  True  True  True]
Time 2 c Early Exercise: [ True  True  True]
Time 1 c Early Exercise: [ True  True]
Time 0 c Early Exercise: [ True]
Time 9 p Early Exercise: [ True  True  True  True  True  True  True  True False False]
Time 8 p Early Exercise: [ True  True  True  True  True  True  True  True False]
Time 7 p Early Exercise: [ True  True  True  True  True  True  True False]
Time 6 p Early Exercise: [ True  True  True  True  True  True  True]
Time 5 p Early Exercise: [ True  True  True  True  True  True]
Time 4 p Early Exercise: 

In [240]:
print(f'Euro @ Zero Rate Put: ${put_euro_r0.round(3)}')
print(f'American @ Zero Rate Put: ${put_american_r0.round(3)}')
print(f'Is american put price equal to euro: {put_american_r0 == put_american_r0}')

Euro @ Zero Rate Put: $28.602
American @ Zero Rate Put: $28.602
Is american put price equal to euro: True


### Problem 2: Option pricing from real data

Note: This problem is the continuation of Prob. 3 in HW #1. We will use the data to
price options and compare with the market prices.

Consider the asset that you selected in HW #1. For this asset, select three call options
and one put option. All options should have approximately the same expiration date, say
about 3 months in the future. The three calls should have three different strike prices,
typically one in-the-money, one at-the-money and one out-of-the-money. (Consider values
relatively close to the current price and, if you have access to this information, for which
the trading volume is relatively large.) The put option should be at-the-money. Record
the current price of these options. Note that all traded options are American options. Also
determine the interest rate to be used for your calculations: typically, you can consider
the LIBOR rate, or the 10-year US treasury yield.


In [187]:
visa = yf.Ticker('V')
v_historical = yf.download('V',start = '2021-01-01', end = '2022-03-03' ).drop(['Open','High','Low','Close'],axis = 1)

[*********************100%***********************]  1 of 1 completed


In [189]:
current_price = v_historical['Adj Close'][-1]
current_price


208.47999572753906

In [129]:
import datetime
from dateutil.relativedelta import relativedelta

def closest_expire_calc(time_delta):
    """
    takes how many months in the future
    returns the closest expire date in future
    """
    target_date = datetime.datetime.today()+relativedelta(months=time_delta)
    dt_expire=[pd.to_datetime(i) for i in visa.options]
    time_diff= []
    for i in range(len(dt_expire)):
        time_diff.append(abs(dt_expire[i] - target_date)) #absolute date difference
    min_diff = min(time_diff)
    min_index = time_diff.index(min_diff)
    return (str(dt_expire[min_index].date()))

In [130]:
expiredate_3_months = closest_expire_calc(3)
opts = visa.option_chain(expiredate_3_months)
v_calls = opts.calls
v_puts = opts.puts

In [318]:
r = yf.download('^tnx',start='2022-03-01')
r = np.round(r['Adj Close'][0],3)
r_dec = r/100
print(f'We use the rate (3/3/2022): {r} %')

[*********************100%***********************]  1 of 1 completed
We use the rate (3/3/2022): 1.707 %


In [177]:
"""Select random in/out/at money calls"""
np.random.seed(10)
in_money_c = v_calls[v_calls['inTheMoney']==True].iloc[np.random.randint(10)]
out_money_c = v_calls[v_calls['inTheMoney']==False].iloc[np.random.randint(10)]
print('\t' +'In the Money Call')
print(in_money_c)
print('\t' +'Out the Money Call')
print(out_money_c)

"""Select Random Put"""
selected_put = v_puts.iloc[np.random.randint(len(v_puts))]
print('\t' +'Selected Put')
print(selected_put)

	In the Money Call
contractSymbol          V220617C00115000
lastTradeDate        2021-11-10 12:00:04
strike                             115.0
lastPrice                          120.5
bid                                 97.2
ask                                100.2
change                               0.0
percentChange                        0.0
volume                                 6
openInterest                          17
impliedVolatility               1.394168
inTheMoney                          True
contractSize                     REGULAR
currency                             USD
Name: 9, dtype: object
	Out the Money Call
contractSymbol          V220617C00220000
lastTradeDate        2022-03-04 16:21:09
strike                             220.0
lastPrice                            6.0
bid                                  6.0
ask                                  6.2
change                             -2.59
percentChange                 -30.151339
volume                              

In [12]:
v_calls[v_calls['strike'].between(200,275)]
price_range = v_historical['Adj Close'].max() - v_historical['Adj Close'].min()
price_vol = v_historical['Adj Close'].std()
price_median = v_historical['Adj Close'].median()
print(f'Price range: {np.round(price_range,2)}, Price Vol: {np.round(price_vol,2)}, Price: Median: {np.round(price_median,2)}')
p_u = 270
p_d = 200
print('u factor is 1.1')
print('d factor is .89')

Price range: 59.89, Price Vol: 12.62, Price: Median: 222.0
u factor is 1.1
d factor is .89



#### (a) We assume that the CRR model represents the stock price and that its assumptions are satisfied. Based on historical data (about 3 months to a year in the past), determine the parameters $\mu$ and $\sigma$ for the stock that you chose following the methods discussed in class. (I recommend to download current data, but feel free to use past data if that is all you have.)


In [ ]:
# mu = 
# sigma = 

#### (b) Price the selected put option using an N-period binomial model for different values of N (which you can choose on your own). As a hint, you can for instance start with $N = 2$ or $N = 3$, which should be approximately monthly-periods. You can then use approximately weekly periods $(N \approx 10,12)$ or daily periods $(N \approx 60)$. Make sure that you use the exact number of days or weeks to the maturity of your option.


#### (c) Compare your results to the market price and comment. In particular, discuss the impact of $N$ on your calculated price.

#### (d) Price each of the three selected call options using all of (i) approximately monthly periods $(N = 2,3)$, (ii) approximately weekly periods $(N \approx 10, 12)$, and (iii) daily periods $(N \approx 60)$.

#### (e) Compare your results to the market prices and comment. In particular, discuss the impact of the strike price K on your results.

In general, once you obtain the parameters $\mu$ and $\sigma$ , you should be able to use the
computing tool that you developed in HW#3 and in Problem 1 to price your options.

### Problem 3

We consider an N-period binomial model with the following properties: each period lasts
a year and the current stock price is $S_0 = 4$. On each period, the stock price doubles
when it moves up and is reduced by half when it moves down. The annual interest rate
on the money market is 25%.


In [212]:
p_3_u = 2
p_3_d =.5
s_0 = 4
r = .25


#### (a) Find the values of $\mu$ and $\sigma$ associated with this CRR model


#### (b) Assuming that $p=\frac{1}{2}$ , find $P(S_T \geq 6)$ when $N = 3$ and $N = 10$

In [265]:
import math
n_3 = 3
n_10 = 10
p=.5
price = s_0* p_3_d **(np.arange(n_10,-1,-1)) * p_3_u ** (np.arange(0,n_10+1,1))

In [326]:
p_4 = math.comb(len(price),math.ceil(len(price)/2))*((p)**math.ceil(len(price)/2))*(1-p)**(len(price)-math.ceil(len(price)/2))
print(f'P(S_T > 6 ) = {round(((1-p_4)/2),4)}')

P(S_T > 6 ) = 0.3872



#### (c) Using the risk-neutral option pricing formula (written as an expectation), calculate the price of a European call with strike price $K = 8$ when $N = 3$ and $N = 10$